# Exploring the data w/ mne

In [2]:
import mne
import numpy as np
import pandas as pd
import keras

In [12]:
epochs = mne.read_epochs("Data/rsvp_session0_files135-142-epo.fif")
eeg_data = epochs.get_data() # NumPy array of shape [n_trials, n_electrodes, n_timepoints] with n_electrodes = 64, n_timepoints = 276 and n_trials is variable by each file.
metadata = epochs.metadata   # Pandas DataFrame

Reading c:\Users\wadem\OneDrive - UCB-O365\EEG2Text\EEG2Text\Clone\Data\rsvp_session0_files135-142-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 64) active
    Found the data of interest:
        t =    -200.00 ...     900.00 ms
        0 CTF compensation matrices available
Adding metadata with 16 columns
6119 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
